In [ ]:
import os
import chainlit as cl
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

model_id = "openchat/openchat_3.5"
conv_model = HuggingFaceHub(huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
                            repo_id=model_id,
                            model_kwargs={"temperature": 0.8, "max_new_tokens": 150})

template = """You are a helpful AI assistant providing support to someone dealing with phobias. Your goal is to offer friendly and informative advice on overcoming fears. 

{query}
"""

@cl.on_chat_start
def main():
    prompt = PromptTemplate(template=template, input_variables=['query'])
    conv_chain = LLMChain(llm=conv_model, prompt=prompt, verbose=True)
    cl.user_session.set("llm_chain", conv_chain)

@cl.on_message
async def process_message(message: str):
    llm_chain = cl.user_session.get("llm_chain")
    res = await llm_chain.acall(message, callbacks=[cl.AsyncLangchainCallbackHandler()])

    # Perform post-processing on the received response here
    # 'res' is a dictionary, and the response text is stored under the key "text"
    return res["text"]

# If you are using Flask
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "Server is running."

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8080)


········


D:\Anaconda\envs\chatbot\Lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


 * Serving Flask app '__main__'
 * Debug mode: off
2023-11-30 03:00:39 - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.31:8080
2023-11-30 03:00:39 - Press CTRL+C to quit
2023-11-30 03:01:09 - 127.0.0.1 - - [30/Nov/2023 03:01:09] "GET / HTTP/1.1" 200 -
2023-11-30 03:01:10 - 127.0.0.1 - - [30/Nov/2023 03:01:10] "GET /favicon.ico HTTP/1.1" 404 -
2023-11-30 03:02:56 - 192.168.1.31 - - [30/Nov/2023 03:02:56] "GET / HTTP/1.1" 200 -
2023-11-30 03:02:56 - 192.168.1.31 - - [30/Nov/2023 03:02:56] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
%run Chatbot.ipynb